# Books Recommender system using clustering

- Dataset :- https://www.kaggle.com/ra4u12/bookrecommendation

In [ ]:
# Importing necessary library
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
books = pd.read_csv('data/BX-Books.csv', sep=";", error_bad_lines=False, encoding='latin-1')

In [ ]:
books.head()

In [ ]:
books.iloc[237]['Image-URL-L']

In [ ]:
books.shape

In [ ]:
books.columns

#### Conclution:
Here Image URL columns is important for the poster. So, we will keep it

In [ ]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [ ]:
books.head()

In [ ]:
# Lets remane some wierd columns name
books.rename(columns={"Book-Title":'title',
                      'Book-Author':'author',
                     "Year-Of-Publication":'year',
                     "Publisher":"publisher",
                     "Image-URL-L":"image_url"},inplace=True)

In [ ]:
books.head()

In [ ]:
# Now load the second dataframe

users = pd.read_csv('data/BX-Users.csv', sep=";", error_bad_lines=False, encoding='latin-1')

In [ ]:
users.head()

In [ ]:
users.shape

In [ ]:
# Lets remane some wierd columns name
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

In [ ]:
users.head(2)

In [ ]:
# Now load the third dataframe

ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=";", error_bad_lines=False, encoding='latin-1')

In [ ]:
ratings.head()

In [ ]:
ratings.shape

In [ ]:
# Lets remane some wierd columns name
ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [ ]:
ratings.head(2)

### Conclution:
Now we have 3 dataframes
- books
- users
- ratings

In [ ]:
print(books.shape, users.shape, ratings.shape, sep='\n')



In [ ]:
ratings['user_id'].value_counts()

In [ ]:
ratings['user_id'].value_counts().shape

In [ ]:
ratings['user_id'].unique().shape

In [ ]:
# Lets store users who had at least rated more than 200 books
x = ratings['user_id'].value_counts() > 200

In [ ]:
x[x].shape

In [ ]:
y= x[x].index

In [ ]:
y

In [ ]:
ratings = ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings.head()

In [ ]:
ratings.shape

In [ ]:
# Now join ratings with books

ratings_with_books = ratings.merge(books, on='ISBN')

In [ ]:
ratings_with_books.head()

In [ ]:
ratings_with_books.shape

In [ ]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
number_rating.head()

In [ ]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [ ]:
number_rating.head()

In [ ]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [ ]:
final_rating.head()

In [ ]:
final_rating.shape

In [ ]:
# Lets take those books which got at least 50 rating of user

final_rating = final_rating[final_rating['num_of_rating'] >= 50]

In [ ]:
final_rating.head()

In [ ]:
final_rating.shape

In [ ]:
# lets drop the duplicates
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [ ]:
final_rating.shape

In [ ]:
# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [ ]:
book_pivot

In [ ]:
book_pivot.shape

In [ ]:
book_pivot.fillna(0, inplace=True)

In [ ]:
book_pivot

# Training Model

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
book_sparse = csr_matrix(book_pivot)

In [ ]:
type(book_sparse)

In [ ]:
# Now import our clustering algoritm which is Nearest Neighbors this is an unsupervised ml algo
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [ ]:
model.fit(book_sparse)

In [ ]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )

In [ ]:
distance

In [ ]:
suggestion

In [ ]:
book_pivot.iloc[241,:]

In [ ]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

In [ ]:
book_pivot.index[3]

In [ ]:
#keeping books name
book_names = book_pivot.index

In [ ]:
book_names[2]

In [ ]:
np.where(book_pivot.index == '4 Blondes')[0][0]

# find url

In [ ]:
# final_rating['title'].value_counts()
ids = np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

In [ ]:
final_rating.iloc[ids]['image_url']

In [ ]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])
    
    

In [ ]:
book_name[0]

In [ ]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [ ]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image_url']
    print(url)

In [ ]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

# Testing model

In [ ]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [ ]:
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)